In [2]:
import os
import openai
import langchain
from langchain.llms import OpenAI

from dotenv import load_dotenv

load_dotenv()

# openai.api_key = os.getenv("OPENAI_API_KEY")

True

In [7]:
llm = OpenAI()

text = "What would be a good company name for a company that makes colorful socks?"

llm.predict(text)

'\n\nRainbowToes Socks.'

In [8]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
    What is a good name for a company
    that makes {product}?
    """
)

prompt.format(product="colorful socks")

'\n    What is a good name for a company\n    that makes colorful socks?\n    '

In [9]:
from langchain.prompts.chat import ChatPromptTemplate

template = 'You are a helpful assistant that translates {input_language} to {output_language}'
human_temaplte = '{text}'

chat_prompt = ChatPromptTemplate.from_messages([
    ('system', template),
    ('human', human_temaplte)
])

chat_prompt.format_messages(input_language='English', output_language="French", text="I love programming.")

[SystemMessage(content='You are a helpful assistant that translates English to French'),
 HumanMessage(content='I love programming.')]

In [19]:
from langchain.schema import BaseOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(', ')

template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

chain = chat_prompt | ChatOpenAI() | CommaSeparatedListOutputParser()
chain.invoke({'text': 'colors'})

['red', 'blue', 'green', 'yellow', 'purple']

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain = prompt | model

In [6]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [7]:
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [ ]:
model.input_schema.schema()

In [12]:
for s in chain.stream({'topic': 'bears'}): 
    print(s.content, end="", flush=True)

Why don't bears use cell phones?
Because they can't bear the cost of roaming charges!

In [13]:
chain.invoke({'topic': 'bears'})

AIMessage(content="Why don't bears wear socks?\n\nBecause they have bear feet!")

In [14]:
chain.batch([{"topic": "bears"}, {"topic": "cats"}])

[AIMessage(content="Why don't bears ever wear shoes?\n\nBecause they have bear feet!"),
 AIMessage(content="Why don't cats play poker in the wild?\n\nToo many cheetahs!")]

In [15]:
from typing import TypedDict, List, Dict, Any, Optional

class LogEntry(TypedDict):
    id: str
    """ID of the sub-run."""
    name: str
    """Name of the object being run."""
    type: str
    """Type of the object being run, eg. prompt, chain, llm, etc."""
    tags: List[str]
    """List of tags for the run."""
    metadata: Dict[str, Any]
    """Key-value pairs of metadata for the run."""
    start_time: str
    """ISO-8601 timestamp of when the run started."""

    streamed_output_str: List[str]
    """List of LLM tokens streamed by this run, if applicable."""
    final_output: Optional[Any]
    """Final output of this run.
    Only available after the run has finished successfully."""
    end_time: Optional[str]
    """ISO-8601 timestamp of when the run ended.
    Only available after the run has finished."""


class RunState(TypedDict):
    id: str
    """ID of the run."""
    streamed_output: List[Any]
    """List of output chunks streamed by Runnable.stream()"""
    final_output: Optional[Any]
    """Final output of the run, usually the result of aggregating (`+`) streamed_output.
    Only available after the run has finished successfully."""

    logs: Dict[str, LogEntry]
    """Map of run names to sub-runs. If filters were supplied, this list will
    contain only the runs that matched the filters."""

In [19]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import FAISS

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

retrieval_chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

async for chunk in retrieval_chain.astream_log(
    "where did harrison work?", include_names=["Docs"]):
    print("-" * 40)
    print(chunk)

----------------------------------------
RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '08eb0a63-3ff9-4b85-8323-0e0aca3732d2',
            'logs': {},
            'streamed_output': []}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/Docs',
  'value': {'end_time': None,
            'final_output': None,
            'id': '5fd4f1d9-2e14-43a2-91fb-1c1772804fba',
            'metadata': {},
            'name': 'Docs',
            'start_time': '2023-11-01T14:15:05.550',
            'streamed_output_str': [],
            'tags': ['map:key:context', 'FAISS', 'OpenAIEmbeddings'],
            'type': 'retriever'}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/Docs/final_output',
  'value': {'documents': [Document(page_content='harrison worked at kensho')]}},
 {'op': 'add',
  'path': '/logs/Docs/end_time',
  'value': '2023-11-01T14:15:05.892'})
------------------------